Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using DynamicHMCModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_d("..", "scripts", "10")
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);
df[:pulled_left] = convert(Array{Int64}, df[:pulled_left])
df[:prosoc_left] = convert(Array{Int64}, df[:prosoc_left])
df[:condition] = convert(Array{Int64}, df[:condition])
df[:actor] = convert(Array{Int64}, df[:actor])
first(df, 5)

struct m_10_04d_model{TY <: AbstractVector, TX <: AbstractMatrix,
  TA <: AbstractVector}
    "Observations."
    y::TY
    "Covariates"
    X::TX
    "Actors"
    A::TA
    "Number of observations"
    N::Int
    "Number of unique actors"
    N_actors::Int
end

Make the type callable with the parameters *as a single argument*.

In [4]:
function (problem::m_10_04d_model)(θ)
    @unpack y, X, A, N, N_actors = problem   # extract the data
    @unpack β, α = θ  # works on the named tuple too
    ll = 0.0
    ll += sum(logpdf.(Normal(0, 10), β)) # bp & bpC
    ll += sum(logpdf.(Normal(0, 10), α)) # alpha[1:7]
    ll += sum(
      [loglikelihood(Binomial(1, logistic(α[A[i]] + dot(X[i, :], β))), [y[i]]) for i in 1:N]
    )
    ll
end

Instantiate the model with data and inits.

In [5]:
N = size(df, 1)
N_actors = length(unique(df[:actor]))
X = hcat(ones(Int64, N), df[:prosoc_left] .* df[:condition]);
A = df[:actor]
y = df[:pulled_left]
p = m_10_04d_model(y, X, A, N, N_actors);
θ = (β = [1.0, 0.0], α = [-1.0, 10.0, -1.0, -1.0, -1.0, 0.0, 2.0])
p(θ)

-305.21943396408915

Write a function to return properly dimensioned transformation.

In [6]:
problem_transformation(p::m_10_04d_model) =
    as( (β = as(Array, size(p.X, 2)), α = as(Array, p.N_actors), ) )
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Tune and sample.

In [7]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.017 s/step ...done
MCMC, adapting ϵ (25 steps)
0.012 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0069 s/step ...done
MCMC, adapting ϵ (100 steps)
0.0065 s/step ...done
MCMC, adapting ϵ (200 steps)
0.0038 s/step ...done
MCMC, adapting ϵ (400 steps)
step 351 (of 400), 0.0029 s/step
0.0028 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0036 s/step ...done
MCMC (1000 steps)
step 310 (of 1000), 0.0032 s/step
step 642 (of 1000), 0.0031 s/step
step 973 (of 1000), 0.0031 s/step
0.0031 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [8]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :α),Tuple{Array{Float64,1},Array{Float64,1}}},1}:
 (β = [5.288535615314311, 0.2899052049665222], α = [-5.61653112393201, 3.5147379335270434, -6.215087320611774, -6.132440758291185, -5.997084863389355, -4.702159054169533, -3.249189936039635])       
 (β = [-1.452543783457585, 0.5658167382269896], α = [1.3690306524406448, 14.436400215346424, 0.04174375010279019, 0.7157810614151905, 0.9087583395745134, 2.0115596124714474, 3.675597263872846])     
 (β = [-4.595485961745888, 0.08614520177255482], α = [4.20733718375844, 19.344466604532258, 3.9803117543543545, 4.07222990178038, 4.155399449774642, 5.067077701527262, 6.555818208512486])           
 (β = [3.042213294318185, 0.477031465205246], α = [-3.4024330375975174, 3.447935551369752, -3.8349606800346825, -3.717682506059053, -3.241356398515965, -2.453147385611382, -0.761185859314911])      
 (β = [2.288100346525088, 0.2363406057298711], α = [-2.6287314144119778, 3.924219603699735, -3.1871298542997044, -3.032999

Extract the parameter posterior means: `β`,

In [9]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 1.4276903024649537 
 0.40544305532375463

Extract the parameter posterior means: `β`,

In [10]:
posterior_α = mean(last, posterior)

7-element Array{Float64,1}:
 -1.865109635638255 
 10.228336505394818 
 -2.1758049836678177
 -2.169447221598316 
 -1.873706211642606 
 -0.936420469772429 
  0.6321095810950799

Effective sample sizes (of untransformed draws)

In [11]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×9 Array{Float64,2}:
 492.208  1000.0  485.799  346.869  …  488.645  491.148  486.632  504.529

NUTS-specific statistics

In [12]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.92, min/25%/median/75%/max: 0.02 0.89 0.96 0.99 1.0
  termination: AdjacentDivergent => 0% AdjacentTurn => 28% DoubledTurn => 72%
  depth: 2 => 1% 3 => 47% 4 => 50% 5 => 1% 6 => 0%


Result rethinking

In [13]:
rethinking = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
        Mean        SD       Naive SE       MCSE      ESS
a.1 -0.74503184 0.26613979 0.0042080396 0.0060183398 1000
a.2 10.77955494 5.32538998 0.0842018089 0.1269148045 1000
a.3 -1.04982353 0.28535997 0.0045119373 0.0049074219 1000
a.4 -1.04898135 0.28129307 0.0044476339 0.0056325117 1000
a.5 -0.74390933 0.26949936 0.0042611590 0.0052178124 1000
a.6  0.21599365 0.26307574 0.0041595927 0.0045153523 1000
a.7  1.81090866 0.39318577 0.0062168129 0.0071483527 1000
 bp  0.83979926 0.26284676 0.0041559722 0.0059795826 1000
bpC -0.12913322 0.29935741 0.0047332562 0.0049519863 1000
";

Means of draws

In [14]:
[posterior_β, posterior_α]

2-element Array{Array{Float64,1},1}:
 [1.4276903024649537, 0.40544305532375463]                                                                                                    
 [-1.865109635638255, 10.228336505394818, -2.1758049836678177, -2.169447221598316, -1.873706211642606, -0.936420469772429, 0.6321095810950799]

End of `10/m10.04d.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*